Theofilus Arkhi Susanto
(theofilus.arkhi@mail.ugm.ac.id)
25/555265/PPA/07030

In [1]:
import numpy as np

In [2]:
matrix = [
    [1, 2, 3, 4],
    [5, 6, 7, 8],
    [9, 0, 1 ,2],
    [3, 4 ,5 ,6]
]

In [3]:
# Downsampling

pool_size = 2

def max_element(matrix, start_y, start_x, pool_size):
  cur_max = -999999999999
  for i in range(start_y, start_y+pool_size):
    for j in range(start_x, start_x+pool_size):
      cur_max = max(cur_max, matrix[i][j])
  return cur_max

def mean_element(matrix, start_y, start_x, pool_size):
  cur_mean = 0
  divisor = 0
  for i in range(start_y, start_y+pool_size):
    for j in range(start_x, start_x+pool_size):
      cur_mean += matrix[i][j]
      divisor += 1
  return cur_mean/max(1,divisor)

def median_element(matrix, start_y, start_x, pool_size):
  median = None
  new_list = []
  for i in range(start_y, start_y+pool_size):
    for j in range(start_x, start_x+pool_size):
      new_list.append(matrix[i][j])
  new_list = sorted(new_list)
  return new_list[(len(new_list)-1)//2]

def max_downsampling(matrix):
  new_height = len(matrix) - 1
  new_width = len(matrix[0]) - 1
  new_list = np.zeros((new_height, new_width), dtype=int)
  for i in range(0, len(matrix)):
    for j in range(0, len(matrix)):
      try:
        new_val = max_element(matrix, i, j, pool_size)
        new_list[i][j]=(new_val)
      except:
        pass

  return new_list

def mean_downsampling(matrix):
  new_height = len(matrix) - 1
  new_width = len(matrix[0]) - 1
  new_list = np.zeros((new_height, new_width), dtype=int)
  for i in range(0, len(matrix)):
    for j in range(0, len(matrix)):
      try:
        new_val = mean_element(matrix, i, j, pool_size)
        new_list[i][j]=(new_val)
      except:
        pass

  return new_list

def median_downsampling(matrix):
  new_height = len(matrix) - 1
  new_width = len(matrix[0]) - 1
  new_list = np.zeros((new_height, new_width), dtype=int)
  for i in range(0, len(matrix)):
    for j in range(0, len(matrix)):
      try:
        new_val = median_element(matrix, i, j, pool_size)
        new_list[i][j]=(new_val)
      except:
        pass

  return new_list

In [4]:
max_downsampling(matrix)

array([[6, 7, 8],
       [9, 7, 8],
       [9, 5, 6]])

In [5]:
mean_downsampling(matrix)

array([[3, 4, 5],
       [5, 3, 4],
       [4, 2, 3]])

In [6]:
median_downsampling(matrix)

array([[2, 3, 4],
       [5, 1, 2],
       [3, 1, 2]])

Nearest neighbor

In [37]:
from collections import deque
class nearest_neighbor:

  def __init__ (self,  matrix):
    self.visited = [[False]*100 for i in range(100)]
    self.lines = [[] for i in range(100)]
    self.matrix = matrix
    self.upsample_matrix = self.generate_upsample_matrix()

  def min_bfs(self, matrix, y, x, dist = 0):
    q=deque()
    q.append((y,x,dist))
    while len(q)!=0:
      self.visited[y][x]=True
      y, x, dist= q.popleft()
      if(y>=len(matrix) or x >=len(matrix[0])):
        continue
      if(y<0 or x<0):
        continue

      if matrix[y][x] != None:
        self.lines[dist].append(matrix[y][x])

      delta_y = [0, 1, 0, -1]
      delta_x = [1, 0, -1, 0]

      for i, _ in enumerate(delta_y):
        new_y = y+delta_y[i]
        new_x = x+delta_x[i]
        if self.visited[new_y][new_x] == False:
          q.append((new_y,new_x,dist+1))

  def nearest_neighbor_min(self):
    new_matrix=np.copy(self.upsample_matrix)
    for i, item_i in enumerate(new_matrix):
      for j, item_j in enumerate(item_i):
        if item_j == None:
          minimum_val = 999999999999
          self.visited = [[False]*100 for i in range(100)]
          self.lines = [[] for i in range(100)]
          self.min_bfs(self.upsample_matrix, i, j)
          for ii in self.lines:
            for jj in ii:
              minimum_val = min(minimum_val, jj)
            if(len(ii)>0):
              break
          new_matrix[i][j]=minimum_val
    return new_matrix

  def generate_upsample_matrix(self):
    new_matrix_height = len(self.matrix)*2-1
    new_matrix_width = len(self.matrix[0])*2-1

    new_matrix = [[None]*new_matrix_width for i in range(new_matrix_height)]

    idx_i_new=0
    for i in range(len(self.matrix)):
      idx_j_new=0
      for j, item_j in enumerate(self.matrix[i]):
        new_matrix[idx_i_new][idx_j_new]=(item_j)
        idx_j_new+=2
      idx_i_new+=2

    return new_matrix



In [39]:
graph = nearest_neighbor(matrix)

In [40]:
np.array(graph.matrix)

array([[1, 2, 3, 4],
       [5, 6, 7, 8],
       [9, 0, 1, 2],
       [3, 4, 5, 6]])

In [42]:
np.array(graph.generate_upsample_matrix())

array([[1, None, 2, None, 3, None, 4],
       [None, None, None, None, None, None, None],
       [5, None, 6, None, 7, None, 8],
       [None, None, None, None, None, None, None],
       [9, None, 0, None, 1, None, 2],
       [None, None, None, None, None, None, None],
       [3, None, 4, None, 5, None, 6]], dtype=object)

In [43]:
graph.nearest_neighbor_min()

array([[1, 1, 2, 2, 3, 3, 4],
       [1, 1, 2, 2, 3, 3, 4],
       [5, 5, 6, 6, 7, 7, 8],
       [5, 0, 0, 0, 1, 1, 2],
       [9, 0, 0, 0, 1, 1, 2],
       [3, 0, 0, 0, 1, 1, 2],
       [3, 3, 4, 4, 5, 5, 6]], dtype=object)

Bilinear

In [51]:
from collections import deque
class bilinear:

  def __init__ (self,  matrix):
    self.visited = [[False]*100 for i in range(100)]
    self.lines = [[] for i in range(100)]
    self.matrix = matrix
    self.upsample_matrix = self.generate_upsample_matrix()

  def bilinear_bfs(self, matrix, y, x, dist = 0):
    q=deque()
    q.append((y,x,dist))
    while len(q)!=0:
      self.visited[y][x]=True
      y, x, dist= q.popleft()
      if(y>=len(matrix) or x >=len(matrix[0])):
        continue
      if(y<0 or x<0):
        continue

      if matrix[y][x] != None:
        self.lines[dist].append(matrix[y][x])

      delta_y = [0, 1, 0, -1]
      delta_x = [1, 0, -1, 0]

      for i, _ in enumerate(delta_y):
        new_y = y+delta_y[i]
        new_x = x+delta_x[i]
        if self.visited[new_y][new_x] == False:
          q.append((new_y,new_x,dist+1))

  def nearest_neighbor_bilinear(self):
    new_matrix=np.copy(self.upsample_matrix)
    for i, item_i in enumerate(new_matrix):
      for j, item_j in enumerate(item_i):
        if item_j == None:
          sum = 0
          dv = 1
          self.visited = [[False]*100 for i in range(100)]
          self.lines = [[] for i in range(100)]
          self.bilinear_bfs(self.upsample_matrix, i, j)
          for ii in self.lines:
            for jj in ii:
              sum += jj
            if(len(ii)>0):
              dv=max(dv,len(ii))
              break
          new_matrix[i][j]=round(sum/dv)
    return new_matrix

  def generate_upsample_matrix(self):
    new_matrix_height = len(self.matrix)*2-1
    new_matrix_width = len(self.matrix[0])*2-1

    new_matrix = [[None]*new_matrix_width for i in range(new_matrix_height)]

    idx_i_new=0
    for i in range(len(self.matrix)):
      idx_j_new=0
      for j, item_j in enumerate(self.matrix[i]):
        new_matrix[idx_i_new][idx_j_new]=(item_j)
        idx_j_new+=2
      idx_i_new+=2

    return new_matrix



In [52]:
bl = bilinear(matrix)

In [53]:
bl.nearest_neighbor_bilinear()

array([[1, 2, 2, 2, 3, 4, 4],
       [3, 4, 4, 4, 5, 6, 6],
       [5, 6, 6, 6, 7, 8, 8],
       [7, 5, 3, 4, 4, 4, 5],
       [9, 4, 0, 0, 1, 2, 2],
       [6, 4, 2, 2, 3, 4, 4],
       [3, 4, 4, 4, 5, 6, 6]], dtype=object)

Bicubic

In [54]:
from collections import deque
class Bicubic:

  def __init__ (self,  matrix):
    self.visited = [[False]*100 for i in range(100)]
    self.lines = [[] for i in range(100)]
    self.matrix = matrix
    self.upsample_matrix = self.generate_upsample_matrix()

  def bicubic_bfs(self, matrix, y, x, dist = 0):
    q=deque()
    q.append((y,x,dist))
    while len(q)!=0:
      self.visited[y][x]=True
      y, x, dist= q.popleft()
      if(y>=len(matrix) or x >=len(matrix[0])):
        continue
      if(y<0 or x<0):
        continue

      if matrix[y][x] != None:
        self.lines[dist].append(matrix[y][x])

      delta_y = [0, 1, 0, -1]
      delta_x = [1, 0, -1, 0]

      for i, _ in enumerate(delta_y):
        new_y = y+delta_y[i]
        new_x = x+delta_x[i]
        if self.visited[new_y][new_x] == False:
          q.append((new_y,new_x,dist+1))

  def nearest_neighbor_bicubic(self):
    new_matrix=np.copy(self.upsample_matrix)
    for i, item_i in enumerate(new_matrix):
      for j, item_j in enumerate(item_i):
        if item_j == None:
          sum = 0
          dv = 0
          self.visited = [[False]*100 for i in range(100)]
          self.lines = [[] for i in range(100)]
          self.bicubic_bfs(self.upsample_matrix, i, j)
          counter=0
          for ii in self.lines:
            for jj in ii:
              sum += jj
            if(len(ii)>0):
              counter+=1
              dv+=len(ii)
            if counter==2:
              break
          new_matrix[i][j]=round(sum/max(1,dv))
    return new_matrix

  def generate_upsample_matrix(self):
    new_matrix_height = len(self.matrix)*2-1
    new_matrix_width = len(self.matrix[0])*2-1

    new_matrix = [[None]*new_matrix_width for i in range(new_matrix_height)]

    idx_i_new=0
    for i in range(len(self.matrix)):
      idx_j_new=0
      for j, item_j in enumerate(self.matrix[i]):
        new_matrix[idx_i_new][idx_j_new]=(item_j)
        idx_j_new+=2
      idx_i_new+=2

    return new_matrix



In [55]:
bc = Bicubic(matrix)

In [57]:
bc.nearest_neighbor_bicubic()

array([[1, 4, 2, 5, 3, 6, 4],
       [4, 4, 4, 4, 5, 4, 5],
       [5, 4, 6, 3, 7, 3, 8],
       [4, 4, 5, 4, 4, 4, 4],
       [9, 4, 0, 5, 1, 5, 2],
       [3, 4, 4, 5, 3, 4, 4],
       [3, 4, 4, 2, 5, 3, 6]], dtype=object)